# Partialing Prompt Templates

Cassandra-powered prompt templates support partialing. Building on the previous examples:

In [1]:
from langchain.prompts import createCassandraPromptTemplate

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
astraSession = getCQLSession()
astraKeyspace = getCQLKeyspace()

ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

cassPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace=astraKeyspace,
    template=ctemplate0,
    input_variables=['city', 'name', 'user_question'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

## Creating a Partial Prompt

Let us "partial" the prompt above, specifying the DB-lookup inputs and leaving only the user question unspecified:

In [3]:
cassPartialPrompt = cassPrompt.partial(city='lisbon', name='Pedro')

When rendering the template, the full lookup takes place at once:

In [4]:
print(cassPartialPrompt.format(user_question='Em verdade, o que quiseres?'))

Please answer a question from a user.
Keep in mind that the user's age is 1 and they live in a city with
nickname ACidade.

USER'S QUESTION: Em verdade, o que quiseres?
YOUR ANSWER:



## Partialing and Chat Prompt Templates

Unfortunately, this is not supported at the LangChain level. Let's create a two-item chat sequence,

In [5]:
systemTemplate = """
You are a chat assistant, helping a user of age {user_age} from a city
they refer to as {city_nickname}.
"""

cassSystemPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace=astraKeyspace,
    template=systemTemplate,
    input_variables=['city', 'name'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
systemMessagePrompt = SystemMessagePromptTemplate(prompt=cassSystemPrompt)

humanTemplate = "{text}"
humanMessagePrompt = HumanMessagePromptTemplate.from_template(humanTemplate)

cassChatPrompt = ChatPromptTemplate.from_messages(
    [systemMessagePrompt, humanMessagePrompt]
)

and try to "partial" it by specifying the DB-lookup parameters:

In [6]:
try:
    cassChatPartialPrompt = cassChatPrompt.partial(
        city='turin',
        name='beppe'
    )
except NotImplementedError:
    print('"NotImplementedError" raised by partialing ChatPromptTemplate')

"NotImplementedError" raised by partialing ChatPromptTemplate
